In [1]:
#Used for displaying plots below the cell
%matplotlib inline
import math

import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

import scipy.stats as stats
from scipy.stats.stats import pearsonr

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

In [21]:
df_geo = pd.read_csv('./geography.csv', sep=',', index_col=0)
df_ram = pd.read_csv('./ram.csv', sep=',', index_col=0)

# Original dataset split for VCS use
df_sales_part1 = pd.read_csv('./sales_ram-part1.csv', sep=',', index_col=0) #The dataset doesn't have a column name. This causes the error.
df_sales_part2 = pd.read_csv('./sales_ram-part2.csv', sep=',', index_col=0)
df_sales = df_sales_part1.append(df_sales_part2)

df_time = pd.read_csv('./time.csv', sep=',', index_col=0)
df_vendor = pd.read_csv('./vendor.csv', sep=',', index_col=0)

/home/avenger/.local/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [22]:
df_sales

,Id,ram_code,time_code,geo_code,vendor_code,sales_uds,sales_currency
2602347,3719,1.0,20130322,25,32,13.749032,10.65
2602348,3719,1.0,20130323,18,32,13.828708,10.65
2602349,3719,1.0,20130326,28,32,13.694297,10.65
2602350,3719,1.0,20130327,25,32,13.690530,10.65
2602351,3719,1.0,20130328,27,32,13.605216,10.65
...,...,...,...,...,...,...,...
6014673,7422,3704.0,20170310,69,47,614.130000,614.13
6014674,7422,3704.0,20170505,75,47,614.130000,614.13
6014675,7422,3704.0,20170510,70,47,614.130000,614.13
6014676,7422,3704.0,20170511,69,47,614.130000,614.13


# Task 1.1: Data understanding

In this section we will go through all the datasets imported and try to mine some interesting information.

## Geography

In [ ]:
df_geo.head()

In [ ]:
print(f"Continents in the dataset: {df_geo['continent'].unique()}")

In [ ]:
print(f"Countries in the dataset: {df_geo['country'].unique()}")

In [ ]:
print(f"Regions in the dataset: {df_geo['region'].unique()}")

In [ ]:
print(f"Currencies in the dataset: {df_geo['currency'].unique()}")

In [ ]:
print(df_geo["geo_code"].duplicated().any())

In [ ]:
df_geo.groupby(["continent","country", "region", "currency"]).size().describe()

Each combination of attributes in the dataset is unique, therefore there are no entries with different geo_code but same content.

In [ ]:
print(df_geo["continent"].isna().any())
print(df_geo["country"].isna().any())
print(df_geo["region"].isna().any())
print(df_geo["currency"].isna().any())

No missing values in sight.

## Ram

In [ ]:
df_ram.head()

In [ ]:
print(f'Ram brands: {df_ram["brand"].unique()}')

In [ ]:
print(f'Ram names: {df_ram["name"].unique()}')

In [ ]:
print(f'Ram memory type: {df_ram["memory_type"].unique()}')

In [ ]:
df_ram.describe()

The values are all within reasonable range, there are no obvious outliers.

In [ ]:
print(df_ram["brand"].isna().any())
print(df_ram["name"].isna().any())
print(df_ram["memory_type"].isna().any())

No missing values.

In [ ]:
df_ram.groupby(["brand", "name", "memory_type", "clock"]).size().describe()

Interestingly there are multiple entries with identical rows (as in brand, name, type of memory and clock) but different ram_code identifiers.  
This might be problematic in case we need to construct the vendor features using the ram_code attribute.  
We will fix the problem in the data preparation section.

In [ ]:
df_ram.corr()

The RAMs included in the dataset do not have a correlation between their clock and memory capacity. 

## Sales

In [ ]:
df_sales.head()

In [ ]:
df_sales.describe()

We will now merge all of the datasets in a single one in order to get a better picture of the sales and the features of the RAMs sold.

In [ ]:
df_sales_merged = df_sales[["Id", "ram_code"]].join(df_ram, on="ram_code", rsuffix="_ram")
df_sales_merged["time_code"] = df_sales["time_code"]
df_sales_merged = df_sales_merged.join(df_time.set_index("time_code"), on="time_code")
df_sales_merged["geo_code"] = df_sales["geo_code"]
df_sales_merged = df_sales_merged.join(df_geo.set_index("geo_code"), on="geo_code")
df_sales_merged["vendor_code"] = df_sales["vendor_code"]
df_sales_merged = df_sales_merged.join(df_vendor.set_index("vendor_code"), on="vendor_code", rsuffix="_vendor")
df_sales_merged = df_sales_merged.join(df_sales[["sales_uds", "sales_currency"]])

In [ ]:
df_sales_merged.head()

In [ ]:
df_sales_merged.dtypes

### Sales_uds meaning
The objective of this section is to understand what sales_uds is.  
The name seems to indicate that it is the value in US dollars of the item.

In [ ]:
df_sales.corr()

It seems that sales_uds and sales_currency are strongly correlated.

In [ ]:
diff_currency_series = (df_sales["sales_uds"]-df_sales["sales_currency"]).abs()
diff_currency_series.describe()

A min value of 0 indicates that there are some rows in which the two values (sales_uds, sales_currency) are identical.  
Let's find out which countries this property holds.

In [ ]:
df_diff_country = df_sales.loc[diff_currency_series == 0].set_index("geo_code").join(df_geo.set_index("geo_code"))
df_diff_country.head()

In [ ]:
df_diff_country["country"].unique()

Given that the only country in the dataset that has sales_usd always equal to sales_currency is the US this confirms our intuition.  
This result is fundamental given that throughout our analysis we will refer to this value in order to have a similar metric across countries.

### Relationship between ram characteristics and price  (TODO: we are missing the memory type, need to take into account)  
In this section we look into the correlation between the features of a RAM module and its price.

In [ ]:
df_sales_merged.corrwith(df_sales_merged["sales_uds"])[["memory", "clock"]]

It seems that there is no correlation between memory/clock characteristics and sales_uds.

In [ ]:
df_sales_merged.groupby("vendor_code")[["memory", "clock"]].corrwith(df_sales_merged["sales_uds"]).plot()
plt.ylabel("Corr")
plt.show()

However, when grouping by vendor_code the correlation between characteristics and price is very strong except for a few vendors.  
In general the clock is not as strongly correlated, positively or negatively, to the price of the ram as it can be seen from the distribution of the correlations.  

Let's try to introduce a temporary feature combining both clock and memory.

In [ ]:
df_test_sales = df_sales_merged.copy()
df_test_sales["ddr_num"] = df_test_sales["memory"] * df_test_sales["clock"]
df_test_sales["MemoryByClock"] = df_test_sales["memory"] * df_test_sales["clock"]

df_test_sales.groupby("vendor_code")[["MemoryByClock"]].corrwith(df_test_sales["sales_uds"]).plot()
plt.ylabel("Corr")
plt.show()

We can see that, except for a few vendors, the majority of has a very strong correlation between the RAM features and its price.  
This is what we expected from the start, now the question becomes why there are vendors that skew this result.

In [ ]:
corr_result = df_test_sales.groupby("vendor_code")[["MemoryByClock"]].corrwith(df_test_sales["sales_uds"])
# We want to take a closer look to the extreme cases
# 0.70 is the cut-off point chosen for a strong correlation
corr_outliers = corr_result.loc[corr_result["MemoryByClock"] < 0.70].sort_values("MemoryByClock").rename(columns={"MemoryByClock":"CorrRamUsd"}) 
corr_outliers.head(10)

In [ ]:
df_outliers_subset = df_test_sales.loc[df_test_sales["vendor_code"].isin(corr_outliers.index.values)]

These are the worst offenders, let's try to find out why.

In [ ]:
corr_subset_result = df_outliers_subset.groupby(["vendor_code", "country", "year"])["MemoryByClock"].corr(df_outliers_subset["sales_uds"])
corr_subset_result

Grouping the vendors sales by (year, country) we can see that there are groups in which the general trend is broken (ie vendor 62, year 2016, UK).  
Let's take a week by week view.

In [ ]:
# 0.70 is the cut-off point chosen for a strong correlation, as done previously
corr_tmp = df_outliers_subset.groupby(["vendor_code", "country", "region", "year"])["MemoryByClock"].corr(df_outliers_subset["sales_uds"])
corr_tmp_res = corr_tmp.loc[corr_tmp < 0.70]

corr_tmp_res

In [ ]:
df_outliers_subset.loc[(df_outliers_subset["vendor_code"] == 72) & (df_outliers_subset["year"] == 2018) & (df_outliers_subset["country"] == "United Kingdom") & (df_outliers_subset["region"] == "yorkshire")]

In [ ]:
df_outliers_subset.loc[(df_outliers_subset["vendor_code"] == 62) & (df_outliers_subset["year"] == 2016)].quantile(0.99)["sales_uds"]

It seems like there are a bunch of "rip-off" purchases going on that are skewing the correlation.  
Removing the outliers should solve the issue.

### Correlation analysis

In [ ]:
clean_sales_df = df_sales_merged.drop(["brand", "name", "memory_type", "continent", "country", "region", "currency", "name_vendor"], axis=1)

In [ ]:
plt.figure(figsize=(15,15))
sn.heatmap(clean_sales_df.corr(), annot=True)
plt.show()

The majority of the correlation coefficients present are not relevant, some however give us some information about the meaning of the data:  
- (ram_code, id): it implies that there is a 1-to-1 relationship between them, in other words for each id we have a single ram_code and viceversa.  
    This makes one of the two attributes redundant which seems strange given that they represent very different things, this is probably connected to the sales dataset duplication previously found and will be analysed shortly.  
- (sales_uds, sales_currency): this correlation is interesting, we have plenty of currencies both weaker and stronger than the us dollars, however we still obtained a value of 1.  
    It signifies that there is a very special pattern here, usefult to groupby currency. (TODO) 
- (week, month): this correlation is a given since month is about week/4.  

### Data semantics

In [ ]:
df_sales.head()

* Id: Id and ram_code are virtually the same, they identify the type of ram sold by the vendor. For each Id there is only a type of ram sold.
* ram_code: see above
* time_code: identifies the date in which the ram was sold by the vendor
* geo_code: identifies the location in which the ram was sold/sent
* vendor_code: identifies the vendor that carried out the transaction
* sales_uds: identifies the value of the sale in us dollars
* sales_currency: identifies the value of the sale in the local currency of the country in which it was sold

## Time

In [ ]:
df_time

In [ ]:
df_time.describe()

No suspicious values here.

In [ ]:
print(f"Duplicated time_code values: {df_time['time_code'].duplicated().any()}")
print(f"Presence of NaN in the dataset: {df_time.isna().any().any()}")

## Vendor

In [ ]:
df_vendor.head()

In [ ]:
df_vendor.describe()

In [ ]:
print(df_vendor["name"].unique())
print(len(df_vendor["name"].unique()))

print(df_vendor["name"].isna().any())
print("" in df_vendor["name"].unique())

In [ ]:
df_vendor.groupby(["vendor_code" ,"name"]).size().transform(lambda x: True if x>1 else False).any()

No duplicates.

# Task 1.2: Data preparation

## Auxiliary functions

In [ ]:
# Note: run %matplotlib notebook to enable interactivity
def plot_3d_scatter(data_points_list, labels_list=None):
    
    if(labels_list):
        if(len(data_points_list) != len(labels_list)):
            raise RuntimeError
        
    list_size = len(data_points_list)
    fig = plt.figure(figsize=(15,10))
    
    for i in range(list_size):
        
        ax = fig.add_subplot((list_size+1)//2, 2, i+1, projection='3d') #row, column, index
        X = data_points_list[i].copy()
        
        if(labels_list):
            
            if(len(data_points_list[i]) != len(labels_list[i])):
                raise RuntimeError

            X["label"] = labels_list[i]

            for l in np.unique(labels_list[i]):
                ax.scatter(X.loc[X.label == l, 0], X.loc[X.label == l, 1], X.loc[X.label == l, 2],
                           cmap=plt.get_cmap("Pastel1"),
                           s=20, edgecolor='k')
        else:
            ax.scatter(X[0], X[1], X[2], s=20, edgecolor='k')

## Missing values

In [ ]:
df_sales_merged.describe()

In [ ]:
df_sales_merged.isna().any()

There doesn't seem to be any missing value, such as NaN or 0.

## Outliers

All the datasets except Sales contain the description of a particular code (ie ram_code), we are therefore gonna assume that there is no point in looking for outliers there since they are just "definitions".  
We will focus on the Sales dataset.

In [ ]:
df_sales.head()

In [ ]:
plt.scatter(df_sales["sales_uds"], df_sales["sales_currency"], color='g', marker='*', label='Standard')
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")
plt.show()

### IQR approach

In [ ]:
Q1 = df_sales.quantile(0.25)
Q3 = df_sales.quantile(0.75)
IQR = Q3 - Q1
right_whisker = Q3 + 1.5*IQR
left_whisker = Q1 - 1.5*IQR

In [ ]:
print(f"Sale USD 2.5-quantile: {df_sales['sales_uds'].quantile(0.025)}")
print(f"Sale USD 97.5-quantile: {df_sales['sales_uds'].quantile(0.975)}")
print(f"Sale Local 2.5-quantile: {df_sales['sales_currency'].quantile(0.025)}")
print(f"Sale Local 97.5-quantile: {df_sales['sales_currency'].quantile(0.975)}\n")

print(f"Left whisker:\n{left_whisker[['sales_uds', 'sales_currency']]}\n")
print(f"Right whisker:\n{right_whisker[['sales_uds', 'sales_currency']]}")

The left whisket is negative. Weird. (TODO?)

In [ ]:
df_wo_out_iqr = df_sales.copy()

df_wo_out_iqr = df_wo_out_iqr.loc[(df_wo_out_iqr["sales_uds"] >= left_whisker["sales_uds"]) & (df_wo_out_iqr["sales_uds"] <= right_whisker["sales_uds"])]
df_wo_out_iqr = df_wo_out_iqr.loc[(df_wo_out_iqr["sales_currency"] >= left_whisker["sales_currency"]) & (df_wo_out_iqr["sales_currency"] <= right_whisker["sales_currency"])]

df_size_diff = df_sales.shape[0]-df_wo_out_iqr.shape[0]
print(f'The number of total outliers in the dataset is: {df_size_diff}')
print(f'That is {df_size_diff*100/df_sales.shape[0]:3.2f}% of the total entries.')

In [ ]:
fig = plt.figure(figsize=(20, 5)) 
fig_dims = (1, 3)
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot2grid(fig_dims, (0, 0))
plt.title("Sampled dataset with outliers")
init_axes = \
    plt.scatter(df_sales["sales_uds"], df_sales["sales_currency"], 
                color="g", marker='*', label='Standard').axes
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.subplot2grid(fig_dims, (0, 1))
plt.title("Sampled dataset without outliers")
plt.scatter(df_wo_out_iqr["sales_uds"], df_wo_out_iqr["sales_currency"], color="g", marker='*', label='Standard')
#Keep the same axis size for all the subplots
plt.xlim(min(init_axes.viewLim.intervalx), max(init_axes.viewLim.intervalx))
plt.ylim(min(init_axes.viewLim.intervaly), max(init_axes.viewLim.intervaly))
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.subplot2grid(fig_dims, (0, 2))
plt.title("Sampled dataset without outliers (Zoomed)")
plt.scatter(df_wo_out_iqr["sales_uds"], df_wo_out_iqr["sales_currency"], color="g", marker='*', label='Standard')
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.show()

This kind of outlier detection doesn't seem suitable to our purpouses since it classifies as outliers a significant amount of points.

## Z-score

In [ ]:
zscore_sample = df_sales[["sales_uds", "sales_currency"]].values

In [ ]:
scaler = StandardScaler()
scaler.fit(zscore_sample)
norm_zscore_sample = scaler.transform(zscore_sample)

In [ ]:
df_zscore = pd.DataFrame(norm_zscore_sample, index=df_sales.index)
df_zscore.describe()

In [ ]:
limBase = df_zscore.std()

fig = plt.figure(figsize=(20, 5)) 
fig_dims = (1, 2)
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot2grid(fig_dims, (0, 0))
plt.title("Max and min values in sales_uds")
plt.xlabel("Number of time the std")
for r in np.arange(0.5, 4, 0.5):
    df_range = df_zscore.loc[((df_zscore[0] > -r*limBase[0]) & (df_zscore[0] < r*limBase[0]))]
    plt_max = df_sales.loc[df_range.index, "sales_uds"].max()
    plt_min = df_sales.loc[df_range.index, "sales_uds"].min()
    plt.scatter(r, plt_min)
    plt.scatter(r, plt_max)
    
plt.subplot2grid(fig_dims, (0, 1))
plt.title("Max and min values in sales_currency")
plt.xlabel("Number of time the std")
for r in np.arange(0.5, 4, 0.5):
    df_range = df_zscore.loc[((df_zscore[1] > -r*limBase[1]) & (df_zscore[1] < r*limBase[1]))]
    plt_max = df_sales.loc[df_range.index, "sales_currency"].max()
    plt_min = df_sales.loc[df_range.index, "sales_currency"].min()
    plt.scatter(r, plt_min)
    plt.scatter(r, plt_max)

plt.show()

In order to simplify the analysis we will only take into account the values in USD and check how reasonable are the max sale values.  
Note: that in the second plot the points values could use different currencies.

In [ ]:
r = 1
df_range = df_zscore.loc[((df_zscore[0] > -r*limBase[0]) & (df_zscore[0] < r*limBase[0]))]
df_sales_merged.loc[df_range.index].sort_values("sales_currency").tail()

While with r=0.5 the maximum values for sales_uds seem to align to a reasonable price for the time in which the sale took place, by increasing the range we get prices that easily double for ram that is specwise half as good (ie ~8000$ for a 8gb stick of ddr4 in 2018).  
This analysis is not ideal since it doesn't take account of the currency exchange nor of the fluctuations in the supply of ram at the time.

In [ ]:
fig = plt.figure(figsize=(20, 5)) 
fig_dims = (1, 2)
fig.subplots_adjust(hspace=0.2, wspace=0.2)

stdTimes = 0.5

plt.subplot2grid(fig_dims, (0, 0))
plt.title("Dataset with outliers")
plt.scatter(df_zscore[0], df_zscore[1], color="g", marker='*', label='Standard')
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.subplot2grid(fig_dims, (0, 1))
plt.title("Dataset without outliers")
df_zscore_wo_out = df_zscore.loc[((df_zscore[0] > -stdTimes*df_zscore[0].std()) & (df_zscore[0] < stdTimes*df_zscore[0].std()))]
#df_zscore_wo_out = df_zscore.loc[((df_zscore[1] > -stdTimes*df_zscore[1].std()) & (df_zscore[1] < stdTimes*df_zscore[1].std()))]
plt.scatter(df_zscore_wo_out[0], df_zscore_wo_out[1], color="g", marker='*', label='Standard')
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.show()

In [ ]:
df_zscore["Outlier"] = 0
df_zscore.loc[~df_zscore.index.isin(df_zscore_wo_out.index), "Outlier"] = 1
df_zscore_outlier_labels = df_zscore["Outlier"]

In [ ]:
df_zscore_out = df_zscore.loc[~df_zscore.index.isin(df_zscore_wo_out.index)]
df_zscore_out.shape[0] #Outliers removed

## Clustering approach

There are too many entries to be able to simply run something like DBSCAN on all of them to detect the noise.  
We will therefore use BIRCH to perform a data reduction and then apply DBSCAN in order to approximate the result.

In [ ]:
#Remove duplicates to speed up clustering/training
cluster_out_sample = df_sales.drop_duplicates(subset=["sales_uds", "sales_currency"])[["sales_uds", "sales_currency"]]

scaler = StandardScaler()
scaler.fit(cluster_out_sample)
norm_out_sample = scaler.transform(cluster_out_sample)

In [ ]:
from sklearn.cluster import Birch
brc = Birch(n_clusters=None, threshold=0.003)
brc.fit(norm_out_sample)

In [ ]:
centers_brc_df = pd.DataFrame(brc.subcluster_centers_)
print(f"Number of CF nodes as a result of Birch: {len(brc.subcluster_labels_)}")

In [ ]:
df_sales_brc = df_sales.copy() #Used to identify all outliers (even duplicates)

sales_sample = df_sales[["sales_uds", "sales_currency"]]

scaler = StandardScaler()
scaler.fit(sales_sample)
norm_out_brc_sample = scaler.transform(sales_sample)

df_sales_brc["BIRCH"] = brc.predict(norm_out_brc_sample)

Let's apply DBSCAN on the BIRCH-produced representative centroids.

In [ ]:
dbscan = DBSCAN(min_samples = 100, eps = 0.3)
dbscan.fit(brc.subcluster_centers_)

cluster_series = pd.Series(data=dbscan.labels_, index=centers_brc_df.index)
cluster_outliers = cluster_series.transform(lambda x: 1 if x == -1 else 0)
centers_brc_df["Outlier"] = cluster_outliers

outlier_brc_df = centers_brc_df.loc[centers_brc_df["Outlier"] == 1]
centers_brc_df_wo_out = centers_brc_df.loc[centers_brc_df["Outlier"] == 0]

outlier_count = list(dbscan.labels_).count(-1)
print(f"Number of outliers (BIRCH): {outlier_count}")
print(f"Percentage of outliers among the data points: {outlier_count/centers_brc_df.shape[0]*100:3.2f}%")
print(f"Silhouette score: {silhouette_score(centers_brc_df, dbscan.labels_)}")
print(f"Number of clusters: {len(np.unique(dbscan.labels_))}")

In [ ]:
fig = plt.figure(figsize=(20, 10)) 
fig_dims = (2, 3)
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot2grid(fig_dims, (0, 0))
plt.title("Sampled dataset with outliers")
init_axes = \
    plt.scatter(centers_brc_df[0], centers_brc_df[1], 
                color="g", marker='*', label='Standard').axes
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.subplot2grid(fig_dims, (0, 1))
plt.title("Sampled dataset without outliers")
plt.scatter(centers_brc_df_wo_out[0], centers_brc_df_wo_out[1], color="g")
#Keep the same axis size for all the subplots
plt.xlim(min(init_axes.viewLim.intervalx), max(init_axes.viewLim.intervalx))
plt.ylim(min(init_axes.viewLim.intervaly), max(init_axes.viewLim.intervaly))
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.subplot2grid(fig_dims, (0, 2))
plt.title("Outliers")
plt.scatter(outlier_brc_df[0], outlier_brc_df[1], color="r")
plt.xlim(min(init_axes.viewLim.intervalx), max(init_axes.viewLim.intervalx))
plt.ylim(min(init_axes.viewLim.intervaly), max(init_axes.viewLim.intervaly))
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.subplot2grid(fig_dims, (1, 1))
plt.title("Sampled dataset without outliers (Zoomed in)")
init_axes2 = plt.scatter(centers_brc_df_wo_out[0], centers_brc_df_wo_out[1], color="g").axes
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.subplot2grid(fig_dims, (1, 0))
plt.title("Sampled dataset with outliers (Zoomed in)")
plt.scatter(centers_brc_df_wo_out[0], centers_brc_df_wo_out[1], color="g")
plt.scatter(outlier_brc_df[0], outlier_brc_df[1], color="r")
plt.xlim(0, 100)
plt.ylim(0, 250)
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.subplot2grid(fig_dims, (1, 2))
plt.title("Outliers (Zoomed in)")
plt.scatter(outlier_brc_df[0], outlier_brc_df[1], color="r")
plt.xlim(0, 100)
plt.ylim(0, 250)
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.show()

In [ ]:
clustering_outlier_df = df_sales_brc.loc[df_sales_brc["BIRCH"].isin(outlier_brc_df.index)].drop(["BIRCH"], axis=1)
cluster_df_wo_out = df_sales_brc.loc[~df_sales_brc["BIRCH"].isin(outlier_brc_df.index)].drop(["BIRCH"], axis=1)

fig = plt.figure(figsize=(20, 5)) 
fig_dims = (1, 3)
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot2grid(fig_dims, (0, 0))
plt.title("Sampled dataset with outliers")
init_axes = \
    plt.scatter(df_sales_brc["sales_uds"], df_sales_brc["sales_currency"], 
                color="g", marker='*', label='Standard').axes
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.subplot2grid(fig_dims, (0, 1))
plt.title("Sampled dataset without outliers")
plt.scatter(cluster_df_wo_out["sales_uds"], cluster_df_wo_out["sales_currency"], color="g", marker='*', label='Standard')
#Keep the same axis size for all the subplots
plt.xlim(min(init_axes.viewLim.intervalx), max(init_axes.viewLim.intervalx))
plt.ylim(min(init_axes.viewLim.intervaly), max(init_axes.viewLim.intervaly))
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.subplot2grid(fig_dims, (0, 2))
plt.title("Outliers")
plt.scatter(clustering_outlier_df["sales_uds"], clustering_outlier_df["sales_currency"], color="r", marker='*', label='Standard')
plt.xlim(min(init_axes.viewLim.intervalx), max(init_axes.viewLim.intervalx))
plt.ylim(min(init_axes.viewLim.intervaly), max(init_axes.viewLim.intervaly))
plt.xlabel("Sales (USD)")
plt.ylabel("Sales (Local currency)")

plt.show()

In [ ]:
df_clustering_wo_out = cluster_df_wo_out
df_clustering_outlier_labels = centers_brc_df["Outlier"]

print(f"Number of outliers in the dataset: {clustering_outlier_df.shape[0]}")

## Conclusion

We will choose the best type of outlier detection depending on the results of the feature analysis.

## Vendor features

In [ ]:
df_sales_merged.head()

### Definition

In [ ]:
def add_features(dest_df, source_df):

    #Modify the data frame locally (possibly not needed)
    source_df = source_df.copy()
    
    #Total number of items sold by vendor
    IFeature = source_df.groupby(["vendor_code"]).size().rename("I")
    dest_df = dest_df.join(IFeature, on="vendor_code")
    
    #Total number of unique items sold by vendor
    IuFeature = source_df.groupby(["vendor_code"]).ram_code.nunique()
    dest_df = dest_df.join(IuFeature, on="vendor_code").rename(columns={"ram_code":"Iu"})
    
    #Max value of item sold (USD)
    MaxValFeature = source_df.groupby(["vendor_code"]).apply(lambda x: x.sales_uds.max()).rename("MaxValuePerOrder")
    dest_df = dest_df.join(MaxValFeature, on="vendor_code")
    
    #Avg value per item sold (USD)
    AvgValFeature = source_df.groupby(["vendor_code"]).apply(lambda x: x.sales_uds.sum()/x.shape[0]).rename("AvgValuePerOrder")
    dest_df = dest_df.join(AvgValFeature, on="vendor_code")
    
    #Max value items sold in a month (USD)
    vendorRamCount = source_df.groupby(["vendor_code", "year", "month"])["sales_uds"].sum()
    IMaxValMonthFeature = vendorRamCount.groupby(["vendor_code"]).max().rename("IMaxMonthSales")
    dest_df = dest_df.join(IMaxValMonthFeature, on="vendor_code")
    
    #Avg value items sold in a month (USD)
    vendorRamCount = source_df.groupby(["vendor_code", "year", "month"])["sales_uds"].sum()
    IAvgValMonthFeature = vendorRamCount.groupby(["vendor_code"]).mean().rename("IAvgMonthSales")
    dest_df = dest_df.join(IAvgValMonthFeature, on="vendor_code")
    
    #Max number items sold in a month
    vendorRamCount = source_df.groupby(["vendor_code", "year", "month"]).size()
    IMaxMonthFeature = vendorRamCount.groupby(["vendor_code"]).max().rename("IMaxMonthItems")
    dest_df = dest_df.join(IMaxMonthFeature, on="vendor_code")
    
    #Avg number items sold in a month
    vendorRamCount = source_df.groupby(["vendor_code", "year", "month"]).size()
    IAvgMonthFeature = vendorRamCount.groupby(["vendor_code"]).mean().rename("IAvgMonthItems")
    dest_df = dest_df.join(IAvgMonthFeature, on="vendor_code")
    
    #Number of months in business
    vendorMonthCount = source_df.groupby(["vendor_code", "year"]).apply(lambda x: x.month.nunique())
    TotMonthFeature = vendorMonthCount.groupby(["vendor_code"]).sum().rename("TotMonthBusiness")
    dest_df = dest_df.join(TotMonthFeature, on="vendor_code")
    
    #The Shannon entropy on the selling behaviour of the vendor: countries of operation
    probSeriesGrouped = source_df.groupby(["vendor_code"])\
        .apply(lambda x: x.groupby(["geo_code"]).size()/x.size)
    logSeriesGrouped = np.log2(probSeriesGrouped)
    ProdProbLogSeriesGrouped = -1 * probSeriesGrouped * logSeriesGrouped
    EFeature = ProdProbLogSeriesGrouped.groupby(["vendor_code"]).sum()
    EFeature.name = "Egeo"
    dest_df = dest_df.join(EFeature, on="vendor_code")

    return dest_df

#### Notes regarding the RAM features  
We deliberately didn't include any features defined on the characteristics of the RAM sold.  
In the context of a vendor dataset we care about the value of the items sold and therefore we assume that the market is more capable of weighting the importance of the RAM features.

Let's apply the features to the datasets with and without outliers.

In [ ]:
unq_vendor_id = df_sales["vendor_code"].sort_values().unique()
vend_df = pd.DataFrame(data=unq_vendor_id, columns=["vendor_code"]).set_index("vendor_code")

df_sales_wo_out_zscore = df_sales_merged.loc[df_zscore_wo_out.index]
df_sales_wo_out_clustering = df_sales_merged.loc[df_clustering_wo_out.index]

#Add the features to the empty dataframe
vend_df_w_out = add_features(vend_df, df_sales_merged) #Dataframe containing customer features with outliers
vend_df_wo_out_zscore = add_features(vend_df, df_sales_wo_out_zscore) #Dataframe containing customer features without outliers (zscore)
vend_df_wo_out_clustering = add_features(vend_df, df_sales_wo_out_clustering) #Dataframe containing customer features without outliers (clustering)

### Feature analysis  
Comparison between feature dataset obtained from the vendor dataset with and without outliers

In [ ]:
vend_df_w_out.head()

#### Correlations

In [ ]:
corr_w_out = vend_df_w_out.corr()
corr_w_out_vis = corr_w_out.copy()
threshold = 0.7

for col in corr_w_out.columns:
    corr_w_out_vis[col] = corr_w_out_vis[col].transform(lambda x: x if abs(x)>threshold else 0)
    
corr_w_out_vis

In [ ]:
corr_wo_out_zscore = vend_df_wo_out_zscore.corr()
corr_wo_out_zscore_vis = corr_wo_out_zscore.copy()
threshold = 0.7

for col in corr_wo_out_zscore.columns:
    corr_wo_out_zscore_vis[col] = corr_wo_out_zscore_vis[col].transform(lambda x: x if abs(x)>threshold else 0)
    
corr_wo_out_zscore_vis

In [ ]:
corr_wo_out_clustering = vend_df_wo_out_clustering.corr()
corr_wo_out_clustering_vis = corr_wo_out_clustering.copy()
threshold = 0.7

for col in corr_wo_out_clustering.columns:
    corr_wo_out_clustering_vis[col] = corr_wo_out_clustering_vis[col].transform(lambda x: x if abs(x)>threshold else 0)
    
corr_wo_out_clustering_vis

In [ ]:
# Difference between the correlation matrixes of the two approaches
diff_corr_outliers = corr_wo_out_zscore-corr_wo_out_clustering

threshold = 0.1

for col in diff_corr_outliers.columns:
    diff_corr_outliers[col] = diff_corr_outliers[col].transform(lambda x: x if abs(x)>threshold else 0)
    
diff_corr_outliers

The difference between features don't change a whole lot when applying either type of technique to remove outliers.

In [ ]:
# Difference between the correlation matrixes of the dataset with and without outliers
diff_corr_outliers = corr_w_out-corr_wo_out_clustering

threshold = 0.3

for col in diff_corr_outliers.columns:
    diff_corr_outliers[col] = diff_corr_outliers[col].transform(lambda x: x if abs(x)>threshold else 0)
    
diff_corr_outliers

The only significant effect that removing the outliers has is regarding some "Max" features.

#### Visualisation

In [ ]:
#With outliers
scaler = StandardScaler()
scaler.fit(vend_df_w_out.values)
norm_out_sample = scaler.transform(vend_df_w_out.values)

#Add name column temporarily
tmp_vendor_df = pd.DataFrame(norm_out_sample, columns=vend_df_w_out.columns)
tmp_vendor_df = tmp_vendor_df.join(df_vendor["name"])

plt.figure(figsize=(15,5))
pd.plotting.parallel_coordinates(tmp_vendor_df, "name")
plt.gca().get_legend().remove() #Remove legend
plt.show()

In [ ]:
#Without outliers (zscore)
scaler = StandardScaler()
scaler.fit(vend_df_wo_out_zscore.values)
norm_out_sample = scaler.transform(vend_df_wo_out_zscore.values)

#Add name column temporarily
tmp_vendor_df = pd.DataFrame(norm_out_sample, columns=vend_df_wo_out_zscore.columns)
tmp_vendor_df = tmp_vendor_df.join(df_vendor["name"])

plt.figure(figsize=(15,5))
pd.plotting.parallel_coordinates(tmp_vendor_df, "name")
plt.gca().get_legend().remove() #Remove legend
plt.show()

In [ ]:
#Without outliers (clustering)
scaler = StandardScaler()
scaler.fit(vend_df_wo_out_clustering.values)
norm_out_sample = scaler.transform(vend_df_wo_out_clustering.values)

#Add name column temporarily
tmp_vendor_df = pd.DataFrame(norm_out_sample, columns=vend_df_wo_out_clustering.columns)
tmp_vendor_df = tmp_vendor_df.join(df_vendor["name"])

plt.figure(figsize=(15,5))
pd.plotting.parallel_coordinates(tmp_vendor_df, "name")
plt.gca().get_legend().remove() #Remove legend
plt.show()

The different outlier detection approaches didn't differ much with respect to their impact on the feature dataset as it can be seen in the above results.  
From now on we will take into account only the clustering approach since it gave about the same results while removing less data points.

In [ ]:
vend_df_wo_out = vend_df_wo_out_clustering

#### Feature redundancy  
For our analysis purposes, we don't need all of the features defined above given the high correlations between them.  
We will therefore remove these redundant features (corr > 0.90) in order to simplify future tasks.

In [ ]:
vend_df_w_out_redund = vend_df_w_out.drop(["IMaxMonthSales", "IAvgMonthSales", "IMaxMonthItems", "IAvgMonthItems"], axis=1)
vend_df_wo_out_redund = vend_df_wo_out.drop(["IMaxMonthSales", "IAvgMonthSales", "IMaxMonthItems", "IAvgMonthItems"], axis=1)

#### Feature outliers  
Analysis of the impact on the feature outliers of keeping or not the outliers in the vendor dataset.

As usual, we first normalise the data in order to apply PCA.

In [ ]:
scaler = StandardScaler()
scaler.fit(vend_df_w_out_redund)
norm_data_w_out = scaler.transform(vend_df_w_out_redund)

In [ ]:
scaler = StandardScaler()
scaler.fit(vend_df_wo_out_redund)
norm_data_wo_out = scaler.transform(vend_df_wo_out_redund)

Let's try again using PCA.

In [ ]:
pca = PCA(n_components=3)
X_pca = pca.fit_transform(norm_data_w_out)
print(f"Variance explained by PCA components: {pca.explained_variance_ratio_.sum()}")
pca_df_w_out = pd.DataFrame(X_pca, index=vend_df_w_out.index)

In [ ]:
pca = PCA(n_components=3)
X_pca = pca.fit_transform(norm_data_wo_out)
print(f"Variance explained by PCA components: {pca.explained_variance_ratio_.sum()}")
pca_df_wo_out = pd.DataFrame(X_pca, index=vend_df_wo_out.index)

Let's apply DBSCAN.

In [ ]:
#With outliers in the initial dataset
outlier_detection = DBSCAN(min_samples = 3, eps = 1.4)
clusters = outlier_detection.fit_predict(pca_df_w_out)
outlier_count = list(clusters).count(-1)
print(f"Number of outliers among the data points: {outlier_count}")
print(f"Percentage of outliers among the data points: {outlier_count/vend_df_w_out.shape[0]*100:3.2f}%")
print(f"Silhouette score: {silhouette_score(pca_df_w_out, outlier_detection.labels_)}")
print(f"Number of clusters: {len(np.unique(outlier_detection.labels_))}")

cluster_series = pd.Series(data=clusters, index=pca_df_w_out.index)
cluster_outliers = cluster_series.transform(lambda x: 1 if x == -1 else 0)

pca_df_w_out["Outlier"] = cluster_outliers
color_list_w_out = pca_df_w_out["Outlier"].map(lambda x: "r" if x == 1 else "g")

In [ ]:
#Without outliers in the initial dataset
outlier_detection = DBSCAN(min_samples = 3, eps = 1.7)
clusters = outlier_detection.fit_predict(pca_df_wo_out)
outlier_count = list(clusters).count(-1)
print(f"Number of outliers among the data points: {outlier_count}")
print(f"Percentage of outliers among the data points: {outlier_count/vend_df_wo_out.shape[0]*100:3.2f}%")
print(f"Silhouette score: {silhouette_score(pca_df_wo_out, outlier_detection.labels_)}")
print(f"Number of clusters: {len(np.unique(outlier_detection.labels_))}")

cluster_series = pd.Series(data=clusters, index=pca_df_wo_out.index)
cluster_outliers = cluster_series.transform(lambda x: 1 if x == -1 else 0)

pca_df_wo_out["Outlier"] = cluster_outliers
color_list_wo_out = pca_df_wo_out["Outlier"].map(lambda x: "r" if x == 1 else "g")

In [ ]:
%matplotlib widget
plot_3d_scatter([pca_df_w_out, pca_df_wo_out], [color_list_w_out, color_list_wo_out])

In [ ]:
out_index_w_out = pca_df_w_out.loc[pca_df_w_out["Outlier"] == 1].index
out_df_w_out = vend_df_w_out_redund.loc[out_index_w_out]

out_df_w_out.join(df_vendor["name"]).head()

In [ ]:
out_index_wo_out = pca_df_wo_out.loc[pca_df_wo_out["Outlier"] == 1].index
out_df_wo_out = vend_df_wo_out_redund.loc[out_index_wo_out]

out_df_wo_out.join(df_vendor["name"]).head()

In [ ]:
clustering_outlier_df.loc[clustering_outlier_df["vendor_code"] == 68]

Removing the outliers in the inital dataset doesn't seem to have much of an impact on the outliers of the feature dataset, the problematic vendors are the same for the most part.  
We prefer however to continue our analysis on the vendor dataset that is built on the sales data without outliers since in some instances, like with the vendor RamCity above, the results can be greatly skewed.  
  
We will continue the analysis with two dataframes: one including the five vendors classified as outliers and one excluding them.  
Both of these datasets have been built using the sales data without outliers.

In [ ]:
feature_df_index = pca_df_wo_out.loc[pca_df_wo_out["Outlier"] == 0].index

feature_df_w_out = vend_df_wo_out_redund
feature_df_wo_out = vend_df_wo_out_redund.loc[feature_df_index]

### Feature analysis  
Comparison between the vendor dataset with and without outliers

In [ ]:
feature_df_w_out.describe()

In [ ]:
feature_df_wo_out.describe()

In [ ]:
#With outliers
scaler = StandardScaler()
scaler.fit(feature_df_w_out.values)
norm_out_sample = scaler.transform(feature_df_w_out.values)

#Add name column temporarily
tmp_vendor_df = pd.DataFrame(norm_out_sample, columns=feature_df_w_out.columns)
tmp_vendor_df = tmp_vendor_df.join(df_vendor["name"])

plt.figure(figsize=(15,5))
plt.title("Vendor dataset (with outliers)")
pd.plotting.parallel_coordinates(tmp_vendor_df, "name")
plt.gca().get_legend().remove() #Remove legend
plt.show()

In [ ]:
#Without outliers
scaler = StandardScaler()
scaler.fit(feature_df_wo_out.values)
norm_out_sample = scaler.transform(feature_df_wo_out.values)

#Add name column temporarily
tmp_vendor_df = pd.DataFrame(norm_out_sample, columns=feature_df_wo_out.columns)
tmp_vendor_df = tmp_vendor_df.join(df_vendor["name"])

plt.figure(figsize=(15,5))
plt.title("Vendor dataset (without outliers)")
pd.plotting.parallel_coordinates(tmp_vendor_df, "name")
plt.gca().get_legend().remove() #Remove legend
plt.show()

#### Correlations

In [ ]:
corr_w_out = feature_df_w_out.corr()
corr_w_out_vis = corr_w_out.copy()
threshold = 0.5

for col in corr_w_out.columns:
    corr_w_out_vis[col] = corr_w_out_vis[col].transform(lambda x: x if abs(x)>threshold else 0)
    
corr_w_out_vis

In [ ]:
corr_wo_out = feature_df_wo_out.corr()
corr_wo_out_vis = corr_wo_out.copy()
threshold = 0.5

for col in corr_wo_out.columns:
    corr_wo_out_vis[col] = corr_wo_out_vis[col].transform(lambda x: x if abs(x)>threshold else 0)
    
corr_wo_out_vis

In [ ]:
diff_corr = corr_wo_out-corr_w_out

threshold = 0.1

for col in diff_corr.columns:
    diff_corr[col] = diff_corr[col].transform(lambda x: x if abs(x)>threshold else 0)
    
diff_corr

### Conclusions  
The two datasets seem to be quite similar, so for now we will keep both of them and compare the results obtained from the clustering and classification tasks.

In [ ]:
feature_df_w_out.to_csv('./task1-result_w_out.csv', sep=',', index=True)
feature_df_wo_out.to_csv('./task1-result_wo_out.csv', sep=',', index=True)

# TODO  
- Fix duplication in ram dataset